# 1 Introduction

# 2 Data Import


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
# Importing the vanilla dataset
df_matches = pd.read_csv('epl-training.csv')

# Importing Data File
df_matches = pd.read_csv('epl-training.csv')

# Ensure the match date is in datetime format
df_matches['Date'] = pd.to_datetime(df_matches['Date'],dayfirst = True)

# Sort the df_matches dataframe by ascending date order
df_matches = df_matches.sort_values(by='Date', ascending=True).reset_index(drop=True)

# Drop any rows where all the values are nan
df_matches = df_matches.dropna()

df_matches.dtypes


In [ ]:
# Travel Distance and Travel Fatigue Index

# Stadium Coordinates
stadium_coordinates = {
    'Swansea': (51.6428, -3.9347),
    'West Ham': (51.5383, -0.0166),
    'Charlton': (51.4865, 0.0368),
    'Wigan': (53.5477, -2.6542),
    'Wolves': (52.5904, -2.1306),
    'Brighton': (50.8609, -0.0801),
    'Bournemouth': (50.7352, -1.8384),
    'Blackpool': (53.8046, -3.0483),
    "Nott'm Forest": (52.9399, -1.1326),
    'Aston Villa': (52.5092, -1.8851),
    'Brentford': (51.4908, -0.2888),
    'Chelsea': (51.4816, -0.1910),
    'Coventry': (52.4481, -1.4956),
    'Sheffield United': (53.3703, -1.4708),
    'Fulham': (51.4749, -0.2216),
    'Leeds': (53.7775, -1.5721),
    'Middlesbrough': (54.5781, -1.2178),
    'Newcastle': (54.9756, -1.6218),
    'Luton': (51.8842, -0.4316),
    'Leicester': (52.6203, -1.1422),
    'Hull': (53.7465, -0.3680),
    'Huddersfield': (53.6543, -1.7684),
    'Southampton': (50.9058, -1.3911),
    'QPR': (51.5093, -0.2322),
    'Bradford': (53.8042, -1.7590),
    'Everton': (53.4387, -2.9662),
    'Blackburn': (53.7286, -2.4894),
    'Man United': (53.4631, -2.2914),
    'Stoke': (52.9884, -2.1754),
    'Reading': (51.4222, -0.9828),
    'Birmingham': (52.4756, -1.8682),
    'Liverpool': (53.4308, -2.9610),
    'Tottenham': (51.6044, -0.0664),
    'Ipswich': (52.0544, 1.1455),
    'Norwich': (52.6221, 1.3091),
    'Watford': (51.6498, -0.4016),
    'Man City': (53.4830, -2.2002),
    'Crystal Palace': (51.3983, -0.0855),
    'Derby': (52.9149, -1.4473),
    'Burnley': (53.7888, -2.2302),
    'Sunderland': (54.9146, -1.3884),
    'West Brom': (52.5090, -1.9639),
    'Arsenal': (51.5549, -0.1084),
    'Portsmouth': (50.7964, -1.0639),
    'Cardiff': (51.4729, -3.2041),
    'Bolton': (53.5805, -2.5357)
}

# Define Haversine function
def haversine(coord1, coord2):
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    return round(km, 3)

# Calculate travel distance
def calculate_travel_distance(row, stadium_coordinates):
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    if home_team in stadium_coordinates and away_team in stadium_coordinates:
        home_coords = stadium_coordinates[home_team]
        away_coords = stadium_coordinates[away_team]
        return haversine(home_coords, away_coords)
    return None  # Handle missing coordinates

# Apply the function to calculate travel distances
df_matches['TravelDistance'] = df_matches.apply(
    calculate_travel_distance, axis=1, args=(stadium_coordinates,)
)

# Normalize Travel Distance to create a Travel Fatigue Index
min_distance = df_matches['TravelDistance'].min()
max_distance = df_matches['TravelDistance'].max()

# Apply Travel Fatigue Index to the Dataframe
df_matches['TravelFatigueIndex'] = (df_matches['TravelDistance'] - min_distance) / (
    max_distance - min_distance
)

# Display a sample of the updated dataset
display(df_matches[['HomeTeam', 'AwayTeam', 'TravelDistance', 'TravelFatigueIndex']].head(10))

In [ ]:
# Importing Manager Information

df_managers = pd.read_excel('PremierLeagueManagers.xlsx')

# Ensure Season_Start and Season_End are in datetime format
df_managers['Season_Start'] = pd.to_datetime(df_managers['Season_Start'],dayfirst = True)
df_managers['Season_End'] = pd.to_datetime(df_managers['Season_End'],dayfirst = True)

# Ensure Season_Start and Season_End are in datetime format
df_managers['Season_Start'] = pd.to_datetime(df_managers['Season_Start'],dayfirst = True)
df_managers['Season_End'] = pd.to_datetime(df_managers['Season_End'],dayfirst = True)

# Function to get the manager for a specific team and match date
def get_manager(team, match_date):
    manager_row = df_managers[
        (df_managers['Club'] == team) &
        (df_managers['Season_Start'] <= match_date) &
        (df_managers['Season_End'] >= match_date)
    ]
    return manager_row['Manager'].iloc[0] if not manager_row.empty else None

# Add HomeManager and AwayManager columns to df_matches
df_matches['HomeManager'] = df_matches.apply(lambda row: get_manager(row['HomeTeam'], row['Date']), axis=1)
df_matches['AwayManager'] = df_matches.apply(lambda row: get_manager(row['AwayTeam'], row['Date']), axis=1)

# Importing Spending Information

df_spending = pd.read_excel('Spending_data.xlsx')

# Ensure Season_Start and Season_End are in datetime format
df_spending['Season_Start'] = pd.to_datetime(df_spending['Season_Start'],dayfirst = True)
df_spending['Season_End'] = pd.to_datetime(df_spending['Season_End'],dayfirst = True)

# Ensure Season_Start and Season_End are in datetime format
df_spending['Season_Start'] = pd.to_datetime(df_spending['Season_Start'],dayfirst = True)
df_spending['Season_End'] = pd.to_datetime(df_spending['Season_End'],dayfirst = True)

# Function to get the expenditure for a specific team and match date
def get_spending(team, match_date):
    spending_row = df_spending[
        (df_spending['Team'] == team) &
        (df_spending['Season_Start'] <= match_date) &
        (df_spending['Season_End'] >= match_date)
    ]
    return spending_row['Expenditure'].iloc[0] if not spending_row.empty else 0

# Add HomeManager and AwayManager columns to df_matches
df_matches['HomeExpenditure'] = df_matches.apply(lambda row: get_spending(row['HomeTeam'], row['Date']), axis=1)
df_matches['AwayExpenditure'] = df_matches.apply(lambda row: get_spending(row['AwayTeam'], row['Date']), axis=1)

# Display a sample of the updated dataset
display(df_matches[['HomeTeam', 'AwayTeam','HomeManager','AwayManager','HomeExpenditure' ,'AwayExpenditure', 'TravelDistance', 'TravelFatigueIndex']].head(10))

df_matches.columns

In [ ]:
# Checking the distribution of FTR Values
print(df_matches['FTR'].value_counts()) # Raw values
print(df_matches['FTR'].value_counts(normalize=True)*100) # Percentage of the results being H, A, or D

# 3 Data Transformation & Exploration

In [ ]:
# Calculating Rolling Average Statistics
k = 38 # Number of matches to look backwards to.

# Function to Calculate Rolling Average Statistics for the Past k Matches. Each row's kAvg does not include the current results.
def generate_kAvg(df, attribute, HomeTeam=True):
    if HomeTeam:
        group_by_team = "HomeTeam"
    elif not HomeTeam:
        group_by_team = "AwayTeam"
    
    kattribute = "k" + attribute
    df[kattribute] = (
    df.groupby(group_by_team)[attribute]     # Group by HomeTeam or AwayTeam
    .transform(lambda x: x.shift(1).rolling(window=k, min_periods=1).mean())  # Shift by 1 to exclude the current match, and create a rolling window of up to k past matches
    )
    df[kattribute] = df[kattribute].fillna(0) # Filling nan values with 0

Home_kAvg_features = ['FTHG','HTHG','HS','HC','HF','HY','HR']
Away_kAvg_features = ['FTAG','HTAG','AS','AC','AF','AY','AR']

for feature in Home_kAvg_features:
    generate_kAvg(df_matches, feature,True)
for feature in Away_kAvg_features:
    generate_kAvg(df_matches, feature,False)
    
display(df_matches)

# Create a new column to store home wins and away wins
df_matches['HomeWin'] = (df_matches['FTR'] == 'H').astype(int)
df_matches['AwayWin'] = (df_matches['FTR'] == 'A').astype(int)

# Calculate rolling win rate for home games
df_matches['HomeWinRate'] = (
    df_matches.groupby('HomeTeam')['HomeWin']    # Group by HomeTeam
    .cumsum()                                    # Cumulative sum of home wins
    / df_matches.groupby('HomeTeam').cumcount()  # Divide by cumulative games played
    .add(1)                                      # To avoid division by zero
)

# Calculate rolling win rate for away games
df_matches['AwayWinRate'] = (
    df_matches.groupby('AwayTeam')['AwayWin']    # Group by AwayTeam
    .cumsum()                                    # Cumulative sum of away wins
    / df_matches.groupby('AwayTeam').cumcount()  # Divide by cumulative games played
    .add(1)                                      # To avoid division by zero
)

# Display relevant columns
display(df_matches)

In [ ]:
n = 5; # Number of matches to look back for this particular home and away pair

def generate_nAvg_pairwise(df, row, attribute):
    # Getting the matches with the current row's home and away team lineup
    matches = df[ (df['HomeTeam'] == row['HomeTeam']) & (df['AwayTeam'] == row['AwayTeam']) ]
    
    # Get the current index of the row
    current_index = row.name
    
    # Get previous matches before this match
    previous_matches = matches[matches.index < current_index]
    
    return (
        previous_matches[attribute]
        .rolling(window=n, min_periods=1)
        .mean()
        .iloc[-1]  # Get the latest rolling average value
        if not previous_matches.empty
        else 0  # Default value for no previous matches
    )
# List of attributes to calculate team pair specific rolling averages for
attributes = ['FTHG','HTHG','HS','HC','HF','HY','HR',
              'FTAG','HTAG','AS','AC','AF','AY','AR']

for attribute in attributes:
    # Defining new columns for the specific combination
    new_column = f"n{attribute}_Pairwise"
    df_matches[new_column] = df_matches.apply(
        lambda row: generate_nAvg_pairwise(df_matches, row, attribute), axis = 1
    )

# Showing what columns are present in the current dataframe
df_matches.columns

In [ ]:
# Team Names
le_teams = LabelEncoder()

# Using the same label encoder for the home and away teams.
le_teams.fit(pd.concat([df_matches['HomeTeam'],df_matches['AwayTeam']])) 

# Using the label encoder to encode the home and away teams
df_matches['HomeTeam_Encoded'] = le_teams.transform(df_matches['HomeTeam'])
df_matches['AwayTeam_Encoded'] = le_teams.transform(df_matches['AwayTeam'])

# FTR and HTR
le_results = LabelEncoder()
le_results.fit(pd.concat([df_matches['FTR'],df_matches['HTR']]))

# Using the label encoder to encode the FTR and HTR
df_matches['FTR_Encoded'] = le_results.transform(df_matches['FTR'])
df_matches['HTR_Encoded'] = le_results.transform(df_matches['HTR'])

# Referee
le_referee = LabelEncoder()
le_referee.fit(df_matches['Referee'])

df_matches['Referee_Encoded'] = le_referee.transform(df_matches['Referee'])

# Managers
le_managers = LabelEncoder() 
le_managers.fit(pd.concat([df_matches['HomeManager'],df_matches['AwayManager']]))
df_matches['HomeManager_Encoded'] = le_managers.transform(df_matches['HomeManager'])
df_matches['AwayManager_Encoded'] = le_managers.transform(df_matches['AwayManager'])

display(df_matches)
df_matches = df_matches.fillna(0)

In [ ]:
# Defining the input features 
input_features = ['HomeTeam_Encoded','AwayTeam_Encoded','HomeManager_Encoded',
                 'AwayManager_Encoded','Referee_Encoded', 'HomeExpenditure', 'AwayExpenditure', 
                  'TravelDistance', 'TravelFatigueIndex',
                  'kFTHG', 'kHTHG','kHS', 'kHC', 'kHF', 'kHY', 
                  'kHR', 'kFTAG', 'kHTAG', 'kAS', 'kAC', 'kAF', 'kAY', 'kAR', 
                  'HomeWinRate', 'AwayWinRate', 
                  'nFTHG_Pairwise', 'nHTHG_Pairwise', 'nHS_Pairwise', 'nHC_Pairwise',
                   'nHF_Pairwise', 'nHY_Pairwise', 'nHR_Pairwise', 'nFTAG_Pairwise',
                   'nHTAG_Pairwise', 'nAS_Pairwise', 'nAC_Pairwise', 'nAF_Pairwise',
                   'nAY_Pairwise', 'nAR_Pairwise']
df_input = df_matches[input_features]
display(df_input)

output_features = ['FTR_Encoded']
df_output = df_matches['FTR_Encoded']
display(df_output)

In [ ]:
#Checking correlation Matrix
correlation_matrix = pd.concat([df_input, df_output], axis=1).corr()

sorted_correlation = correlation_matrix['FTR_Encoded'].abs().sort_values(ascending = False)

display(sorted_correlation)

In [ ]:
# Drop the Referee_Encoded column
df_input = df_input.drop('Referee_Encoded', axis = 1)
input_features.remove("Referee_Encoded")

# Display the finalised input feature list
display(df_input)

# 4 Methodology Overview

In [ ]:
#Splitting the dataset into training and test data
from sklearn.model_selection import train_test_split
testing_size = 0.33
input_train, input_test, output_train, output_test = train_test_split(df_input, df_output, test_size=testing_size, random_state=42)

In [ ]:
# Scaling the input training and test data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

input_train_scaled = scaler.fit_transform(input_train)
input_test_scaled = scaler.transform(input_test)


# 5 Model Training & Validation

# 6 Results

# 7 Final Predictions on Test Set